## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv('../Weather_Database/WeatherPy_Database.csv')
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Big Spring,US,32.25,-101.48,93.20,10,1,13.87,clear sky
1,1,Esperance,AU,-33.87,121.90,55.40,100,0,4.70,clear sky
2,2,Wanning,CN,18.80,110.40,78.21,85,100,16.31,overcast clouds
3,3,Punta Arenas,CL,-53.15,-70.92,48.20,57,20,10.29,few clouds
4,4,Bethel,US,41.37,-73.41,59.00,47,90,4.70,overcast clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 85


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Wanning,CN,18.80,110.40,78.21,85,100,16.31,overcast clouds
6,6,Rikitea,PF,-23.12,-134.97,74.17,77,8,10.80,clear sky
7,7,Butaritari,KI,3.07,172.79,81.03,74,23,12.73,few clouds
10,10,Trairi,BR,-3.28,-39.27,77.92,81,0,9.46,clear sky
12,12,Rosetta,EG,31.40,30.42,73.40,60,0,4.70,clear sky
13,13,Sola,VU,-13.88,167.55,84.20,83,75,9.17,broken clouds
14,14,Mineiros,BR,-17.57,-52.55,82.17,40,74,8.59,moderate rain
16,16,Acopiara,BR,-6.10,-39.45,80.60,47,0,9.51,clear sky
19,19,Babu,CN,24.42,111.52,69.33,77,4,4.45,clear sky
22,22,Sao Joao Da Barra,BR,-21.64,-41.05,69.93,63,99,5.55,overcast clouds


In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                321
City                   321
Country                321
Lat                    321
Lng                    321
Max Temp               321
Humidity               321
Cloudiness             321
Wind Speed             321
Current Description    321
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.replace("", np.nan, inplace=True)
preferred_cities_df.dropna(inplace=True)

C:\Users\whitn\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-19-3c230f64140c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preferred_cities_df.dropna(inplace=True)


In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Wanning,CN,78.21,overcast clouds,18.80,110.40,
6,Rikitea,PF,74.17,clear sky,-23.12,-134.97,
7,Butaritari,KI,81.03,few clouds,3.07,172.79,
10,Trairi,BR,77.92,clear sky,-3.28,-39.27,
12,Rosetta,EG,73.40,clear sky,31.40,30.42,
13,Sola,VU,84.20,broken clouds,-13.88,167.55,
14,Mineiros,BR,82.17,moderate rain,-17.57,-52.55,
16,Acopiara,BR,80.60,clear sky,-6.10,-39.45,
19,Babu,CN,69.33,clear sky,24.42,111.52,
22,Sao Joao Da Barra,BR,69.93,overcast clouds,-21.64,-41.05,


In [16]:
hotel_df.count()

City                   321
Country                321
Max Temp               321
Current Description    321
Lat                    321
Lng                    321
Hotel Name             321
dtype: int64

In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found...skipping.')

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [20]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace("", np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)

# Rename the DataFrame as clean_hotel_df
clean_hotel_df = hotel_df
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Wanning,CN,78.21,overcast clouds,18.80,110.40,Paradise Hotel
6,Rikitea,PF,74.17,clear sky,-23.12,-134.97,Pension Maro'i
7,Butaritari,KI,81.03,few clouds,3.07,172.79,Isles Sunset Lodge
10,Trairi,BR,77.92,clear sky,-3.28,-39.27,Pousada Field Lilies - Trairi
12,Rosetta,EG,73.40,clear sky,31.40,30.42,Green Valley
...,...,...,...,...,...,...,...
689,Salta,AR,69.80,clear sky,-24.79,-65.41,Hotel Alejandro I
690,Ipora,BR,72.77,light rain,-16.44,-51.12,Oásis Motel
692,Lethem,GY,82.85,broken clouds,3.38,-59.80,Savannah Inn
693,Islahiye,TR,73.99,clear sky,37.02,36.63,İslahiye Sabanci Teachers


In [21]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))